In [60]:
import pandas as pd
import os
import py_entitymatching as em
import numpy as np

datasets_dir = os.getcwd() + os.sep

pathA = datasets_dir + "/data/imdb_clean.csv"
pathB = datasets_dir + "/data/tomato_clean.csv"
pathC = datasets_dir + "/data/block.csv"

A = pd.read_csv(pathA)
B = pd.read_csv(pathB)
# Rename first empty attr 
# df.rename(columns={"Unnamed: 0": "id"},  inplace=True)

p_A = A[['movie_no', 'movie_name', 'movie_year', 'movie_director', 'movie_star']]
p_B = B[['movie_no', 'movie_name', 'movie_year', 'movie_director', 'movie_star']]

em.set_key(p_A, 'movie_no')
em.set_key(p_B, 'movie_no')

pathS = datasets_dir + "/data/labeled_data.csv"

S = em.read_csv_metadata(pathS, 
                         key='_id',
                         ltable=p_A, rtable=p_B, 
                         fk_ltable='ltable_movie_no', fk_rtable='rtable_movie_no')

IJ = em.split_train_test(S, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [62]:
# Classifier
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NB')

In [63]:
# Feature generation
F = em.get_features_for_matching(p_A, p_B, validate_inferred_attr_types=False)

H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='label',
                            show_progress=False)

# Missing value
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_movie_no', 'rtable_movie_no', 'label'],
                strategy='mean')

In [64]:
# Corss Validation 
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_movie_no', 'rtable_movie_no', 'label'],
        k=5, # Num of fold
        target_attr='label', metric_to_select_matcher='f1', random_state=0)

In [66]:
# Apply to testing set

fc = result['selected_matcher'] # LinReg here
# dt = em.DTMatcher(name='DT', random_state=0)

fc.fit(table=H, 
       exclude_attrs=['_id', 'ltable_movie_no', 'rtable_movie_no', 'label'], 
       target_attr='label')

L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='label', show_progress=False)

predictions = fc.predict(table=L, exclude_attrs=['_id', 'ltable_movie_no', 'rtable_movie_no', 'label'], 
              append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 98.18% (54/55)
Recall : 96.43% (54/56)
F1 : 97.3%
False positives : 1 (out of 55 positive predictions)
False negatives : 2 (out of 98 negative predictions)


In [68]:
# C is the blokced data 
C = em.read_csv_metadata(pathC,
                        key='_id',
                        ltable=p_A, rtable=p_B,
                        fk_ltable='ltable_movie_no', fk_rtable='rtable_movie_no')

cl = result['selected_matcher']

L = em.extract_feature_vecs(C, feature_table=F,
                            show_progress=False)


predictions = cl.predict(table=L, exclude_attrs=['_id', 'ltable_movie_no', 'rtable_movie_no'], 
              append=True, target_attr='predicted', probs_attr='score', return_probs = True, inplace=False)


In [124]:
# Tuple predicted as match
tn = predictions[(predictions['predicted'] == 1)]
C[C['_id'].isin(tn['_id'])]

,_id,ltable_movie_no,rtable_movie_no,ltable_movie_name,ltable_movie_director,ltable_movie_star,rtable_movie_name,rtable_movie_director,rtable_movie_star
0,106,1454,1,annie,Will Gluck,"Quvenzhan Wallis, Cameron Diaz, Jamie Foxx, Rose Byrne",annie,Will Gluck,"Jamie Foxx, Rose Byrne, Bobby Cannavale, Cameron Diaz, Adewale Akinnuoye-Agbaje, David Zayas, Zo..."
3,3391,2450,6,the family,Luc Besson,"Robert De Niro, Michelle Pfeiffer, Dianna Agron, John D'Leo",the family,Luc Besson,"Robert De Niro, Michelle Pfeiffer, Tommy Lee Jones, Dianna Agron, John D'Leo, Kresh Novakovic"
4,4410,2917,9,love beats rhymes,RZA,"Common, Hana Mae Lee, Arjun Gupta, Lucien Laviscount",love beats rhymes,RZA,"Azealia Banks, Jill Scott, Lucien Laviscount, Lorraine Toussaint, Hana Mae Lee, John David Washi..."
5,5260,135,13,john wick,"Chad Stahelski, David Leitch","Keanu Reeves, Michael Nyqvist, Alfie Allen, Willem Dafoe",john wick,"David Leitch, Chad Stahelski","Keanu Reeves, Michael Nyqvist, Alfie Allen, Adrianne Palicki, Bridget Moynahan, Dean Winters, Ia..."
7,6113,306,17,the revenant,Alejandro G. Irritu,"Leonardo DiCaprio, Tom Hardy, Will Poulter, Domhnall Gleeson",the revenant,Alejandro González Iñárritu,"Leonardo DiCaprio, Tom Hardy, Domhnall Gleeson, Will Poulter, Forrest Goodluck, Paul Anderson (X..."
21,7832,1042,21,despicable me 2,"Pierre Coffin, Chris Renaud","Steve Carell, Kristen Wiig, Benjamin Bratt, Miranda Cosgrove",despicable me 2,"Pierre Coffin, Chris Renaud","Steve Carell, Kristen Wiig, Benjamin Bratt, Miranda Cosgrove, Russell Brand, Ken Jeong, Al Pacin..."
22,7940,172,22,batman v superman: dawn of justice,Zack Snyder,"Ben Affleck, Henry Cavill, Amy Adams, Jesse Eisenberg",batman v superman: dawn of justice,Zack Snyder,"Ben Affleck, Henry Cavill, Amy Adams, Diane Lane, Laurence Fishburne, Gal Gadot, Jesse Eisenberg..."
23,8193,2712,23,a little chaos,Alan Rickman,"Kate Winslet, Alan Rickman, Stanley Tucci, Matthias Schoenaerts",a little chaos,Alan Rickman,"Kate Winslet, Matthias Schoenaerts, Alan Rickman, Stanley Tucci, Helen McCrory, Jennifer Ehle, T..."
24,8298,1792,24,wild,Jean-Marc Valle,"Reese Witherspoon, Laura Dern, Gaby Hoffmann, Michiel Huisman",wild,Jean-Marc Vallée,"Reese Witherspoon, Thomas Sadoski, Michiel Huisman, W. Earl Brown, Gaby Hoffmann, Kevin Rankin, ..."
25,8305,2244,25,their finest,Lone Scherfig,"Gemma Arterton, Sam Claflin, Bill Nighy, Richard E. Grant",their finest,Lone Scherfig,"Gemma Arterton, Bill Nighy, Sam Claflin, Jack Huston, Jake Lacy, Richard E. Grant, Paul Ritter, ..."


In [125]:
# Duplicate detect
dup_A = tn[tn["ltable_movie_no"].duplicated(keep=False)]
dup_B = tn[tn["rtable_movie_no"].duplicated(keep=False)]
# C[C['_id'].isin(dup_A['_id'])] 
# C[C['_id'].isin(dup_B['_id'])] 

In [126]:
dup_T = pd.merge(dup_A, dup_B, how='outer', on=['_id','ltable_movie_no', 'rtable_movie_no'])
dup_T

In [127]:
# Remove duplicate
# clean_dup_A = tn["ltable_movie_no"].drop_duplicates(keep='first')
# clean_dup_B = tn["rtable_movie_no"].drop_duplicates(keep='first')
clean_dup_A = tn[~(tn["ltable_movie_no"].isin(dup_T['ltable_movie_no']))]
clean_dup_B = tn[~(tn["rtable_movie_no"].isin(dup_T['rtable_movie_no']))]
# clean_dup_B

In [128]:
# Remove all matched from A and B (A' and B')
new_A = A[(~A['movie_no'].isin(clean_dup_A['ltable_movie_no']))]
new_B = B[(~B['movie_no'].isin(clean_dup_B['rtable_movie_no']))]

In [129]:
# Select one pair of duplicate pairs
dict_A = {}
dict_B = {}

res = []
s_A = []
s_B = []
for index, row in dup_T.iterrows():
    if (row['ltable_movie_no'] not in dict_A) and (row['rtable_movie_no'] not in dict_B):
        dict_A[row['ltable_movie_no']] = 1
        dict_B[row['rtable_movie_no']] = 1
        s_A.append(row['ltable_movie_no'])
        s_B.append(row['rtable_movie_no'])
        res.append(row['_id'])

dict_RA = {}    
dict_RB = {}
for index, row in dup_T.iterrows():
    if (row['ltable_movie_no'] not in dict_A):
        dict_RA[row['ltable_movie_no']] = 1;
    if (row['rtable_movie_no'] not in dict_B):
        dict_RB[row['rtable_movie_no']] = 1;

In [157]:
dict_RB
rem_B = B[B['movie_no'].isin(dict_RB)]

dict_RA
rem_A = A[A['movie_no'].isin(dict_RA)]

In [177]:
# s_A = clean_dup_A['ltable_movie_no'].append(pd.Series(s_A))
# s_B = clean_dup_B['rtable_movie_no'].append(pd.Series(s_B))
rem_B

,movie_name,movie_year,movie_certificate,movie_runtime,movie_genre,movie_score,movie_gross,movie_director,movie_star,movie_writer,tomatoter,audience
1961,the disaster artist,2017,R,104 minutes,Comedy,NaN,NaN,James Franco,"James Franco, Seth Rogen, Dave Franco, Josh Hutcherson, Alison Brie, Zac Efron, Kate Upton, Jack...","Scott Neustadter, Michael H. Weber",91%,88%
2611,the theory of everything,2014,PG-13,123 minutes,Drama,NaN,"$35,887,263",James Marsh,"Eddie Redmayne, Felicity Jones, Emily Watson, Charlie Cox, David Thewlis, Simon McBurney, Maxine...",Anthony McCarten,78%,84%


In [133]:
m_A = A[(A['movie_no'].isin(clean_dup_A['ltable_movie_no']))]
m_B = B[(B['movie_no'].isin(clean_dup_B['rtable_movie_no']))]

In [134]:
clean_dup_A.rename(columns={"ltable_movie_no": "movie_no"}, inplace = True)
clean_dup_B.rename(columns={"rtable_movie_no": "movie_no"}, inplace = True)

m_B = pd.merge(m_B, clean_dup_B[['_id', 'movie_no']], how = 'left', on = 'movie_no')
m_A = pd.merge(m_A, clean_dup_A[['_id', 'movie_no']], how = 'left', on = 'movie_no')

In [135]:
m_F = pd.merge(m_A[['_id','movie_no', 'movie_name', 'movie_year', 'movie_certificate', 'movie_runtime', 
                   'movie_genre', 'movie_score', 'movie_gross', 'movie_director', 'movie_star']],
               m_B[['_id','movie_writer', 'tomatoter', 'audience']], how = 'left', on = '_id')

In [136]:
m_F

,_id,movie_no,movie_name,movie_year,movie_certificate,movie_runtime,movie_genre,movie_score,movie_gross,movie_director,movie_star,movie_writer,tomatoter,audience
0,301372,6,jumanji: welcome to the jungle,2017,PG-13,119 min,"Action, Adventure, Comedy",7.0,$403.11M,Jake Kasdan,"Dwayne Johnson, Karen Gillan, Kevin Hart, Jack Black","Chris McKenna, Erik Sommers, Scott Rosenberg, Jeff Pinkner",76%,88%
1,133732,11,the shape of water,2017,R,123 min,"Adventure, Drama, Fantasy",7.5,$63.54M,Guillermo del Toro,"Sally Hawkins, Octavia Spencer, Michael Shannon, Doug Jones","Guillermo del Toro, Vanessa Taylor",92%,75%
2,29027,13,thor: ragnarok,2017,PG-13,130 min,"Action, Adventure, Comedy",7.9,$315.06M,Taika Waititi,"Chris Hemsworth, Tom Hiddleston, Cate Blanchett, Mark Ruffalo",Eric Pearson,92%,87%
3,47783,17,justice league,2017,PG-13,120 min,"Action, Adventure, Fantasy",6.7,$229.02M,Zack Snyder,"Ben Affleck, Gal Gadot, Jason Momoa, Ezra Miller","Chris Terrio, Joss Whedon",40%,76%
4,92321,22,coco,2017,PG,105 min,"Animation, Adventure, Comedy",8.5,$209.54M,"Lee Unkrich, Adrian Molina","Anthony Gonzalez, Gael Garca Bernal, Benjamin Bratt, Alanna Ubach","Matthew Aldrich, Adrian Molina",97%,95%
5,95187,23,"three billboards outside ebbing, missouri",2017,R,115 min,"Crime, Drama",8.2,$54.25M,Martin McDonagh,"Frances McDormand, Woody Harrelson, Sam Rockwell, Caleb Landry Jones",Martin McDonagh,92%,87%
6,583385,26,pitch perfect 3,2017,PG-13,93 min,"Comedy, Music",6.0,$104.90M,Trish Sie,"Anna Kendrick, Rebel Wilson, Brittany Snow, Anna Camp","Mike White, Kay Cannon",32%,49%
7,356040,32,murder on the orient express,2017,PG-13,114 min,"Crime, Drama, Mystery",6.6,$102.83M,Kenneth Branagh,"Kenneth Branagh, Penlope Cruz, Willem Dafoe, Judi Dench",Michael Green,57%,56%
8,491461,34,ferdinand,2017,PG,108 min,"Animation, Adventure, Comedy",6.7,$84.14M,Carlos Saldanha,"John Cena, Kate McKinnon, Bobby Cannavale, Jack Gore","Robert L. Baird, Tim Federle, Brad Copeland",71%,49%
9,227165,38,"i, tonya",2017,R,120 min,"Biography, Comedy, Drama",7.6,$29.95M,Craig Gillespie,"Margot Robbie, Sebastian Stan, Allison Janney, Julianne Nicholson",Steven Rogers,90%,89%


In [150]:
m_Dl = tn[tn["_id"].isin(res)]
m_Dr = m_Dl.copy()
m_Dl.rename(columns={"ltable_movie_no": "movie_no"}, inplace = True)
m_Dr.rename(columns={"rtable_movie_no": "movie_no"}, inplace = True)

In [151]:
m_DA = A[(A['movie_no'].isin(s_A))]
m_DB = B[(B['movie_no'].isin(s_B))]

In [154]:
m_DB = pd.merge(m_DB, m_Dr[['_id', 'movie_no']], how = 'left', on = 'movie_no')
m_DA = pd.merge(m_DA, m_Dl[['_id', 'movie_no']], how = 'left', on = 'movie_no')

In [155]:
m_DF = pd.merge(m_DA[['_id','movie_no', 'movie_name', 'movie_year', 'movie_certificate', 'movie_runtime', 
                   'movie_genre', 'movie_score', 'movie_gross', 'movie_director', 'movie_star']],
               m_DB[['_id','movie_writer', 'tomatoter', 'audience']], how = 'left', on = '_id')

In [167]:
new_A = new_A.drop(['Unnamed: 0', 'movie_no'], axis = 1)
new_B = new_B.drop(['Unnamed: 0', 'movie_no'], axis = 1)
rem_A = rem_A.drop(['Unnamed: 0', 'movie_no'], axis = 1)
rem_B = rem_B.drop(['Unnamed: 0', 'movie_no'], axis = 1)
m_F = m_F.drop(['_id', 'movie_no'], axis=1)
m_DF = m_DF.drop(['_id', 'movie_no'], axis=1)

In [178]:
frames = [new_A, rem_A, new_B, rem_B, m_F, m_DF]

In [179]:
result = pd.concat(frames, ignore_index=True)

In [180]:
result

,movie_name,movie_year,movie_certificate,movie_runtime,movie_genre,movie_score,movie_gross,movie_director,movie_star,movie_writer,tomatoter,audience
0,ready player one,2018,PG-13,140 min,"Action, Adventure, Sci-Fi",7.9,$68.35M,Steven Spielberg,"Tye Sheridan, Olivia Cooke, Ben Mendelsohn, Lena Waithe",NaN,NaN,NaN
1,avengers: infinity war,2018,NaN,156 min,"Action, Adventure, Fantasy",NaN,NaN,"Anthony Russo, Joe Russo","Karen Gillan, Elizabeth Olsen, Josh Brolin, Scarlett Johansson",NaN,NaN,NaN
2,pacific rim: uprising,2018,PG-13,111 min,"Action, Adventure, Sci-Fi",6.0,$49.21M,Steven S. DeKnight,"John Boyega, Scott Eastwood, Cailee Spaeny, Burn Gorman",NaN,NaN,NaN
3,black panther,2018,PG-13,134 min,"Action, Adventure, Sci-Fi",7.8,$655.62M,Ryan Coogler,"Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o, Danai Gurira",NaN,NaN,NaN
4,tomb raider,2018,PG-13,118 min,"Action, Adventure",6.8,$52.73M,Roar Uthaug,"Alicia Vikander, Dominic West, Walton Goggins, Daniel Wu",NaN,NaN,NaN
5,"game over, man!",2018,TV-MA,101 min,"Action, Comedy",5.4,NaN,Kyle Newacheck,"Adam Devine, Anders Holm, Blake Anderson, Chloe Bridges",NaN,NaN,NaN
6,the greatest showman,2017,PG,105 min,"Biography, Drama, Musical",7.8,$172.49M,Michael Gracey,"Hugh Jackman, Michelle Williams, Zac Efron, Zendaya",NaN,NaN,NaN
7,annihilation,2018,R,115 min,"Adventure, Drama, Fantasy",7.1,$32.31M,Alex Garland,"Natalie Portman, Jennifer Jason Leigh, Tessa Thompson, Benedict Wong",NaN,NaN,NaN
8,a quiet place,2018,PG-13,90 min,"Drama, Horror, Thriller",8.3,NaN,John Krasinski,"Emily Blunt, John Krasinski, Millicent Simmonds, Noah Jupe",NaN,NaN,NaN
9,isle of dogs,2018,PG-13,101 min,"Animation, Adventure, Comedy",8.2,$7.16M,Wes Anderson,"Bryan Cranston, Koyu Rankin, Edward Norton, Bob Balaban",NaN,NaN,NaN


In [181]:
result.to_csv("merge_table.csv", encoding='utf-8')